In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import cv2
from PIL import Image, ImageTk
import numpy as np
import os

# Art Categories
categories = {
    "Traditional Art": "img/Traditional/",
    "Modern Art": "img/Modern/",
    "Contemporary Art": "img/Contemporary/",
}

# FUnctions for enhancement
def adjust_brightness_contrast(image, alpha, beta):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

def sharpen_image(image, strength):
    kernel = np.array([[0, -strength, 0], 
                       [-strength, 1 + 4*strength, -strength],
                       [0, -strength, 0]])
    return cv2.filter2D(image, -1, kernel)

def reduce_noise(image, ksize):
    return cv2.GaussianBlur(image, (ksize, ksize), 0)

def apply_arithmetic(image, operation, value):
    value_matrix = np.ones(image.shape, dtype=np.uint8) * value
    return cv2.add(image, value_matrix) if operation == 'add' else cv2.subtract(image, value_matrix)

def resize_image(image, width, height):
    return cv2.resize(image, (width, height))

def inverse_transform(image):
    return cv2.bitwise_not(image)

def color_space_adjustment(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#Code too cluttered ..so placeholder. I'm here to help you login
def login_screen():
    for widget in root.winfo_children():
        widget.destroy()
    
    tk.Label(root, text="Username:").pack()
    tk.Entry(root, textvariable=username).pack()
    
    tk.Label(root, text="Password:").pack()
    tk.Entry(root, textvariable=password, show='*').pack()
    
    tk.Button(root, text="Login", command=select_category).pack()

def select_category():
    if not username.get() or not password.get():
        messagebox.showerror("Error", "Username and Password missing!.")
        return
    
    for widget in root.winfo_children():
        widget.destroy()
    
    tk.Label(root, text=f"Welcome, {username.get()}! We are so happy you're interested in art. Please select a category to explore:").pack()
    
    for category in categories.keys():
        tk.Button(root, text=category, command=lambda c=category: select_image(c)).pack()
    
    tk.Button(root, text="Back", command=login_screen).place(x=10, y=10)
    tk.Button(root, text="Logout", command=login_screen).place(x=720, y=10)


def select_image(category):
    global selected_category
    selected_category = category
    folder = categories[category]
    
    for widget in root.winfo_children():
        widget.destroy()
    
    tk.Label(root, text=f"Select an image from {category}:").pack()
    
    images = [f for f in os.listdir(folder) if f.endswith(('.jpg', '.png', '.jpeg'))]
    for img in images:
        img_path = os.path.join(folder, img)
        image = Image.open(img_path)
        image.thumbnail((100, 100))
        img_tk = ImageTk.PhotoImage(image)
        btn = tk.Button(root, image=img_tk, command=lambda i=img_path: load_image(i))
        btn.image = img_tk
        btn.pack()
    
    tk.Button(root, text="Back", command=select_category).place(x=10, y=10)
    tk.Button(root, text="Logout", command=login_screen).place(x=720, y=10)


def load_image(path):
    global selected_image_path
    selected_image_path = path
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    image.thumbnail((300, 300))
    
    for widget in root.winfo_children():
        widget.destroy()
    
    img_tk = ImageTk.PhotoImage(image)
    tk.Label(root, image=img_tk).pack()
    root.image_display = img_tk
    
    tk.Button(root, text="Enhance Image", command=select_enhancement).pack()
    tk.Button(root, text="Back", command=lambda: select_image(selected_category)).place(x=10, y=10)
    tk.Button(root, text="Logout", command=login_screen).place(x=720, y=10)


def select_enhancement():
    for widget in root.winfo_children():
        widget.destroy()
    
    tk.Label(root, text="Select an enhancement technique:").pack()
    
    enhancements = {
        
        "Arithmetic Operations": apply_arithmetic_operations,
        
        "Brightness & Contrast": apply_brightness_contrast,
        
        "Color Space Adjustment": apply_color_space_adjustment,
        
        "Inverse Transform": apply_inverse_transform,
        
        "Noise Reduction": apply_noise_reduction,
        
        "Resize Image": apply_resize,
        
        "Sharpening": apply_sharpening,
    }
    
    for name, func in enhancements.items():
        tk.Button(root, text=name, command=func).pack()
    
    tk.Button(root, text="Back", command=lambda: load_image(selected_image_path)).place(x=10, y=10)
    tk.Button(root, text="Logout", command=login_screen).place(x=720, y=10)


def apply_brightness_contrast():
    alpha = simpledialog.askfloat("Brightness & Contrast", "Enter contrast (default 1.2):", minvalue=0.5, maxvalue=3.0) or 1.2
    beta = simpledialog.askinteger("Brightness & Contrast", "Enter brightness (default 30):", minvalue=-100, maxvalue=100) or 30
    process_image(adjust_brightness_contrast, alpha, beta)

def apply_sharpening():
    strength = simpledialog.askfloat("Sharpening", "Enter sharpening strength (default 1.0):", minvalue=0.1, maxvalue=5.0) or 1.0
    process_image(sharpen_image, strength)

def apply_noise_reduction():
    ksize = simpledialog.askinteger("Noise Reduction", "Enter kernel size (odd number, default 5):", minvalue=3, maxvalue=15) or 5
    process_image(reduce_noise, ksize)

def apply_arithmetic_operations():
    operation = simpledialog.askstring("Arithmetic", "Enter operation (add/subtract):")
    value = simpledialog.askinteger("Arithmetic", "Enter value (default 30):", minvalue=0, maxvalue=255) or 30
    process_image(apply_arithmetic, operation, value)

def apply_resize():
    width = simpledialog.askinteger("Resize", "Enter new width:")
    height = simpledialog.askinteger("Resize", "Enter new height:")
    process_image(resize_image, width, height)

def apply_inverse_transform():
    process_image(inverse_transform)

def apply_color_space_adjustment():
    process_image(color_space_adjustment)

def process_image(func, *args):
    image = cv2.imread(selected_image_path)
    enhanced = func(image, *args)
    cv2.imshow("Enhanced Image", enhanced)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    select_enhancement()

root = tk.Tk()
root.title("YSMA Art Enhancer")
root.geometry("800x600")
username = tk.StringVar()
password = tk.StringVar()
selected_category = None
selected_image_path = None

# logo = Image.open("img/logo.jpg")  
# logo = ImageTk.PhotoImage(logo) 
# root.iconphoto(False, logo)  

login_screen()
root.mainloop()


ImportError: DLL load failed while importing cv2: The specified module could not be found.